This is a minimalistic demonstration to transform the [minimum convex cost problem to find the most likely payment flow on the Lightning network](https://arxiv.org/abs/2107.05322) to a [piecewise linearized](https://en.wikipedia.org/wiki/Piecewise_linear_function) problem that can be solved in sub second time on the current channel graph with the help of a [linear min cost flow solver](https://developers.google.com/optimization/reference/graph/min_cost_flow). 

## Main idea and mathematical background of piece wise linearization
Using the ideas of [probabilistic payment delivery](https://arxiv.org/abs/2103.08576) (sometimes known as probabilistic path finding) which has been already [implemented](https://github.com/ElementsProject/lightning/pull/4771) and [tested by c-lightning](https://medium.com/blockstream/c-lightning-v0-10-2-bitcoin-dust-consensus-rule-33e777d58657) as well as [implemented by LDK](https://github.com/lightningdevkit/rust-lightning/pull/1227) (aka rust lightning) we know that the cost function to assign to assign the amount $a$ to a channel of capacity $c$ to use when selecting channels should be 

$f_c(a) = -\log\left(\frac{c+1-a}{c+1}\right)$ 

The linear approximation of this cost function can be found by looking at the first term of the [Taylor Series](https://en.wikipedia.org/wiki/Taylor_series) which turns out to be: 

$l_c(a) = \frac{a}{c}$

The linearized term is easy to compute and interprete: The cost of using the channel is $0$ if no amount is assigned to it and $1$ if the channel is fully saturated. For all other amounts the cost is just proportional to the fraction of saturation. (We note that this term (even thout we started to approximate the negativ log of the successprobability) is also the failure probabliity of a payment)

However just using the linearized version yields two problems in practise: 

1. the unit cost $\frac{1}{c}$ is a float and not an integer (**making it hard for many mcf solving algorithms**!)
2. The linear nature of the problem (like the linear feerate) tends to fully saturate cheap paths which from a reliablity perspective is a very poor choice as fully saturated channels have the lowest probability to be successfull.

To mitigate the first problem with floating values as unit costs we multiply all unit costs with $C_{max}$ as the max capacity that is observed on the channel graph. This will just be a linear scaling of the global cost function and thus not change the solution that minimizes the the cost objective. Of course we still have to round down to integers to make integer unit costs. The function will look like: 

$L_c(a) = a\cdot\lfloor\frac{C_{max}}{c}\rfloor$


To mitigate the second problem instead of using a single linearized cost function on the entire channel we split the channel in $N$ sements (in our case of equal size to demonstrate a point about runtime. (From an approximation perspective one might want to use the optimal piecewise linear approximation which can also be found via: http://www.iaeng.org/publication/WCECS2008/WCECS2008_pp1191-1194.pdf). So when building the linear approximation of the **uncertainty network** instead of adding one channel with capacity $c$ for each channel we add $N$ channels each of capacity $\frac{c}{N}$. The unit cost of the i-th piecewise linearized channel (with $i\in\{0,...,N-1\}$) increases via the following formula: 

$L_{c,i}(a) = L_{c}(a)\cdot(i+1)$

### Motivation of this choice for the cost function on the piecewise linear segments
When using a linear min cost flow solver the unit cost can be seen as the derrivative of the cost function. with the formular $L_{c,i}(a) = L_{c}(a)\cdot(i+1)$ the unit cost is linearly increasing of every segment of the channel. This effectively behaves like piecewise approximation of a quadratic cost function (the derrivative of a quadratic function grows linearly as our adopted piecewise lienar cost function). 

Of course in practise one would approximate the derrivative of the negative log probabilities for the boundaries of the piecewise segments and probably one would not use segments of equal size.

As this code is to demonstrate feasability of runtime (the linearized model of the **uncertainty network** on which we calculates has $N$ times as many edges as the convex problem) this very pragmatic and easy to be implemented choice will be good enough. More work needs to be conducted to make a proper piecewise linear approximation. For example it seems very logical to split of the certain part of the liquidity to have capacity of the certain liquidity and cost 0 (which corresponds to probability 1) but we leave this for future and engineering when implementing into nodes, wallets or LSPs.

## Run the script:
you will need to have a copy of the channel graph from c-lightning in json format. you can get it via:

    $ lightning-cli listchannels > listchannels20211028.json
    
Also you need to have python and jupyther (for example via anaconda) running and [Google OR-tools installed](https://developers.google.com/optimization/install) via

    python -m pip install --upgrade --user ortools


## Warning: This code DOES NOT 
* include optimization for routing fees (in the case of prallel channels it does not even account the paid fees properly)
* include the round based algorithm on the uncertainty network which learns conditional probabilities from attempted onions (check our Paper or [this github comment](https://github.com/lightningdevkit/rust-lightning/issues/1170#issuecomment-972396747) to learn how to do this)
* include the disection of the flow into paths (which is conceptionally straight forward)
* care for HTLCs limits, channel reserves and the like (as all of that is more engineering level)
* Use the optimal piecewise linear approximation for the convex cost function (as described here: http://www.iaeng.org/publication/WCECS2008/WCECS2008_pp1191-1194.pdf) 
* properly handle parallel public channels (actually it just virtually combines the capacity which from a probabilistic perspective makes a hell lot of sense)
* include a simluation of the round based algorithm - in particular no assumptions about actual liquidity are made even the channels of the starting node are assumed to have uncertain liquidity (which in reality is always wrong)
* make any mainnet test payments
* give a guarantee of how close the approximation is to the actual optimal (I am pretty confident when using proper optimal piecewise linearization one can also guarantee the error of the flow of this approximation)

## Funding and Acknowledgements
This research result is funded through [NTNU](https://www.ntnu.no) and [BitMEX](https://blog.bitmex.com/bitmex-2021-open-source-developer-grants/) as well as generous donations from the Bitcoin community via https://donate.ln.rene-pickhardt.de and via recurring donations at https://www.patreon.com/renepickhardt. If you want to learn why independet research and development for Bitoin and the Lightning Network is important I kindly refere you to: https://ln.rene-pickhardt.de/ of course I will be grateful if you consider my work of importance and decide to support it

Special Thanks to Stefan Richter and Carsten Otto for helpful discussions which lead me to the realization that the linearization itself was not the problem of my initial attempts to quickly compute an approximation of the problem but rather the usage of floating point unit costs which seem to be tricky even for linear solvers (yes math still surprises and amazes me)

In [1]:
import json
import time

# using googles linear min cost flow solver as and externaly for convenience.
# it seems to use a cost scaling algorithm internally find more information 
# on their API doc at:  https://developers.google.com/optimization/reference/graph/min_cost_flow
from ortools.graph import pywrapgraph


Next we set a few global variables. Global because our entire code is basically one script with exactly $100$ lines of code (assuming I couted correctly which is hard!) to demonstrate the simplicity and idea of the fast approximation without distraction of boiler plate code.

In [2]:
#to map node_ids to the range [0,...,#number of nodes] and vice versa
node_key_to_id = {}
id_to_node_key = {}


#the will become the list of arcs that are actually stored in the 
arcs = []

#used to store the capacity of channels
channel_graph = {}

#used to store fees as a touple (base_fee_msat,ppm)
fee_graph = {}

We set a few parameters for the experiment. Some of these numbers might heavily impact runtime.

In [3]:
#quantizing payments sets a lower bound on sent HTLCs and speeds up the computation a bit
#set this to 1 if it is to be turned off
QUANTIZATION = 10000

#renes node
SRC = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
#loop node
DEST = "021c97a90a411ff2b10dc2a8e32de2f29d2fa49d41bfbb52bd416e460db0747d0d"
AMT = 50*1000*1000 # 0.5 Bitcoin

#number of piecewise linear approximations. 
#Increasing this directly increases runtime but also improves accuracy
N = 5

## This is where the magic happens

towards the end of the import function the piece wise linear approximation of the cost function takes place and is thus where the magic happens

In [4]:
def import_channels():
    """
    this does all the magic! it imports the channel_graph from c-lightning listchannels command
    
    it first passes through the channels to find all node ids and max capacity
    in a second pass it goes over all channels and adds arcs to the modelled linearized network
    for each channel N arcs are being added with increasing unit costs to mimick convex behavior
    the piecewise dissection is not optimal nor is the linear approximation of negative log probs exact
    however this does not matter for the sake of argument the runtime will not change if the costs
    are chosen abit bit more optimally. But the code will blow up thus those simplifications
    """
    #$ lightning-cli listchannels > listchannels20211028.json
    f = open("listchannels20211028.json")
    channels = json.load(f)["channels"]

    # let's first find the max channel capacity and all node_ids 
    # so that we can build the look up table and use integer unit costs
    max_cap = 0
    node_ids = set()
    for c in channels:
        src = c["source"]
        dest = c["destination"]
        node_ids.add(src)
        node_ids.add(dest)
        cap = c["satoshis"]
        u = 1.0/cap
        #FIXME: ignores the later used virtual combination of channels capacities into one large channel
        if cap>max_cap:
            max_cap = cap
    
    print("Max capacity is: ", max_cap)
    
    # let's initialize the look up tables for node_ids to integers from [0,...,#number of nodes]
    for k, node_id in enumerate(node_ids):
        node_key_to_id[node_id]=k
        id_to_node_key[k]=node_id

    
    # initilize global channel_graph and fee_graph data structures 
    global channel_graph
    channel_graph={node_key_to_id[n]:{} for n in node_ids}
    global fee_graph
    fee_graph={node_key_to_id[n]:{} for n in node_ids}

    global arcs
    arcs = []
    for c in channels:
        src = node_key_to_id[c["source"]]
        dest = node_key_to_id[c["destination"]]
        cap = c["satoshis"]
        
        # we put channels into channel_Graph data structure
        # in case of parallel channels we combine capacity into 1 channel
        # from a probabilistic point of view (which we are interested in) this is correct
        if dest in channel_graph[src]:
            channel_graph[src][dest]+=cap
        else:
            channel_graph[src][dest]=cap
            
        # FIXME: this ignores fees of paralel channels. Ok for us as fees are not our main concern here
        fee_graph[src][dest] = (c["base_fee_millisatoshi"],c["fee_per_millionth"])

        unit_cost = int(max_cap/cap)
        #recall: N is the number of piecewise linear approximations of our cost function
        #FIXME: use optimal linear approximation e.g.: http://www.iaeng.org/publication/WCECS2008/WCECS2008_pp1191-1194.pdf
        # so for each channel we add N arcs with c/N capacity and increasing unit cost to mimick convex nature
        for i in range(N):
            #arc format is src, dest, capacity, unit_cost
            # THIS IS THE IMPORTANT LINE OF CODE WHERE THE MAGIC HAPPENS
            arcs.append((src,dest,int(cap/(N*QUANTIZATION)),(i+1)*unit_cost))

import_channels()

Max capacity is:  1400000000


## Invoking the min cost flow solver
Now that we have created the model of the linearized uncertainty network we have to plug this into a linear min cost flow solver. The following code is basically and adoption of the example at google operation research API doc which can be found at https://developers.google.com/optimization/flow/mincostflow

In order to do so we first need to put all prepared arcs with piecewise linearized integer unit cost to the SimpleMinCostFlow solver.

In [5]:
# Instantiate a SimpleMinCostFlow solver.
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

# Add each of the prepared arcs from import_channels().
for arc in arcs:
    min_cost_flow.AddArcWithCapacityAndUnitCost(arc[0], arc[1], arc[2],
                                                arc[3])

# Add node supply to 0 for all nodes
for i in id_to_node_key.keys():
    min_cost_flow.SetNodeSupply(i, 0)
    
#add amount to sending node
min_cost_flow.SetNodeSupply(node_key_to_id[SRC],int(AMT/QUANTIZATION))

#add -amount to recipient nods
min_cost_flow.SetNodeSupply(node_key_to_id[DEST],-int(AMT/QUANTIZATION))

Only put the execution of the solver into run time computation. Building of the uncertainty network can be done while channels are announced on gossip as the arcs do in practise not change - unless one actively maintains the uncertainty network with knowledge from successfull and failed attempts. This is also cheap in practise as it just updates a few edges for each attempted onion.

In [6]:
start = time.time()
status = min_cost_flow.Solve()
end = time.time()

if status != min_cost_flow.OPTIMAL:
    print('There was an issue with the min cost flow input.')
    print(f'Status: {status}')
    exit(1)

In order to have propper probability computation of the flow we need to combine the assigned flow of the piecewise linearized arcs back to a single flow value for each channel from the original channel graph 

In [7]:
total_flow = {}

#first collect all linearized edges which are assigned a non zero flow
for i in range(min_cost_flow.NumArcs()):
    if min_cost_flow.Flow(i) == 0:
        continue
    cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
    src = min_cost_flow.Tail(i)
    dest = min_cost_flow.Head(i)
    flow = min_cost_flow.Flow(i)*QUANTIZATION

    key = str(src)+":"+str(dest)
    if key in total_flow:
        total_flow[key]=(src,dest,total_flow[key][2]+flow)
    else:
        total_flow[key]=(src,dest,flow)

## Let us print the results

We first define two helper functions for depicting the results. We want to be able to compute the actual probability of the flow and we want to also be able to know what the flow (if fully successfull) would cost

In [8]:
def uniform_probability(a,s,d):
    """
    Computes the uniform probablity of a payment of amout `a` on a channel s-->d
    """
    c = channel_graph[s][d]
    return float(c+1-a)/(c+1)

def fee_msat(a,s,d):
    """
    Computes the the fees of a payment of amout `a` on a channel s-->d
    """
    base, rate = fee_graph[s][d]
    # note we divide ppm by 1000 to be compatible with base_fee wich is measured in msat and not sats
    return base + a*rate/1000

Finally we print out all the results so one can have a nice look at them

In [9]:
print("Planning to deliver {:4.2f} BTC from {}({}...) to {} ({}...) via an approximated optimally reliable payment flow...\n"
      .format(AMT/100./1000/1000, node_key_to_id[SRC], SRC[:7],node_key_to_id[DEST], DEST[:7]))

print("Runtime of flow computation: {:4.2f} sec ".format(end-start))

print('Minimum approximated quadratic cost: ', min_cost_flow.OptimalCost(),"\n")
#print('')
print(' Arc \t\t\t      Flow / Capacity \tprobability \tFee (sats)')

total_fee = 0
probability = 1

#print all edges and compute total fees and total probability of the computed flow
for k,flow_value in total_flow.items():
    src,dest,flow = flow_value
    channel_probability=uniform_probability(flow,src,dest)
    fee = fee_msat(flow,src,dest)
    total_fee += fee/1000
    print('%1s -> %1s     \t  %3s / %3s \t%3f\t%3f' %
          (src, dest,flow, channel_graph[src][dest], channel_probability,fee / 1000))
    probability *= channel_probability
print("\nProbability of entire flow: {:6.4f}".format(probability))
print("Total fee: {} sats \nEffective fee rate: {:5.3f} %".format(total_fee,total_fee*100./AMT))
print("Arcs included in payment flow:", len(total_flow))

print("\nDon't get confused by a low probability. The first attampt always has high uncertainty. We will learn fast in each consequitive round.")


Planning to deliver 0.50 BTC from 2418(03efccf...) to 2982 (021c97a...) via an approximated optimally reliable payment flow...

Runtime of flow computation: 0.78 sec 
Minimum approximated quadratic cost:  815932 

 Arc 			      Flow / Capacity 	probability 	Fee (sats)
2418 -> 11548     	  6700000 / 16777215 	0.600649	8957.900000
2418 -> 13279     	  1800000 / 9000000 	0.800000	2406.600000
2418 -> 9341     	  1240000 / 6200000 	0.800000	1657.880000
2418 -> 1972     	  6700000 / 16777215 	0.600649	8957.900000
2418 -> 9267     	  2000000 / 10000000 	0.800000	2674.000000
5613 -> 9267     	  3350000 / 2411344242 	0.998611	335.100000
9267 -> 683     	  690000 / 6200000000 	0.999889	1.690000
7434 -> 5613     	  3350000 / 20000000 	0.832500	34.500000
1972 -> 683     	  6700000 / 100000000 	0.933000	3350.000000
5975 -> 2982     	  6700000 / 200000000 	0.966500	33501.000000
2418 -> 7821     	  3300000 / 15000000 	0.780000	4412.100000
14825 -> 3976     	  6700000 / 1000000000 	0.993300	2010.00000